In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
import pandas._testing as tm
import mlflow
import wandb

In [14]:
data = yf.download("BTC-USD, GC=F, ^GSPC, ^IXIC, ^VIX", start="2020-01-02", end="2022-01-01")
data.columns

[*********************100%***********************]  5 of 5 completed


MultiIndex([('Adj Close', 'BTC-USD'),
            ('Adj Close',    'GC=F'),
            ('Adj Close',   '^GSPC'),
            ('Adj Close',   '^IXIC'),
            ('Adj Close',    '^VIX'),
            (    'Close', 'BTC-USD'),
            (    'Close',    'GC=F'),
            (    'Close',   '^GSPC'),
            (    'Close',   '^IXIC'),
            (    'Close',    '^VIX'),
            (     'High', 'BTC-USD'),
            (     'High',    'GC=F'),
            (     'High',   '^GSPC'),
            (     'High',   '^IXIC'),
            (     'High',    '^VIX'),
            (      'Low', 'BTC-USD'),
            (      'Low',    'GC=F'),
            (      'Low',   '^GSPC'),
            (      'Low',   '^IXIC'),
            (      'Low',    '^VIX'),
            (     'Open', 'BTC-USD'),
            (     'Open',    'GC=F'),
            (     'Open',   '^GSPC'),
            (     'Open',   '^IXIC'),
            (     'Open',    '^VIX'),
            (   'Volume', 'BTC-USD'),
            

In [15]:
df = data[['Volume', "Close"]] # Оставляем данные: Объем и дневная Цена закрытия
df1 = df.drop([('Volume','^VIX')], axis=1) # Убираем колонку "Объем" "Волатильность" т.к. отсутствуют данные

In [26]:
# Испытал способ формирования данных посредством конкатенации (вышеобозначенный способ проще)
a = data[['Volume']]
b = data[["Close"]]
x = pd.concat([a, b], axis=1)
x

Volume                                                 Close  \
                BTC-USD   GC=F         ^GSPC         ^IXIC ^VIX       BTC-USD   
Date                                                                            
2020-01-01  18565664997    NaN           NaN           NaN  NaN   7200.174316   
2020-01-02  20802083465  214.0  3.458250e+09  2.848370e+09  0.0   6985.470215   
2020-01-03  28111481032  107.0  3.461290e+09  2.567400e+09  0.0   7344.884277   
2020-01-04  18444271275    NaN           NaN           NaN  NaN   7410.656738   
2020-01-05  19725074095    NaN           NaN           NaN  NaN   7411.317383   
...                 ...    ...           ...           ...  ...           ...   
2021-12-27  24324345758  150.0  2.264120e+09  3.730120e+09  0.0  50640.417969   
2021-12-28  33430376883  146.0  2.217050e+09  3.623600e+09  0.0  47588.855469   
2021-12-29  30049226299  623.0  2.369370e+09  3.694500e+09  0.0  46444.710938   
2021-12-30  26686491018  306.0  2.390990e+09  3.732730e+09  0.0  47178.125000   
2021-12-31  36974172400   80.0  2.446190e+09  3.379850e+09  0.0  46306.445312   

                                                               
                   GC=F        ^GSPC         ^IXIC       ^VIX  
Date                                                           
2020-01-01          NaN          NaN           NaN        NaN  
2020-01-02  1524.500000  3257.850098   9092.190430  12.470000  
2020-01-03  1549.199951  3234.850098   9020.769531  14.020000  
2020-01-04          NaN          NaN           NaN        NaN  
2020-01-05          NaN          NaN           NaN        NaN  
...                 ...          ...           ...        ...  
2021-12-27  1808.099976  4791.189941  15871.259766  17.680000  
2021-12-28  1810.199951  4786.350098  15781.719727  17.540001  
2021-12-29  1805.099976  4793.060059  15766.219727  16.950001  
2021-12-30  1812.699951  4778.729980  15741.559570  17.330000  
2021-12-31  1827.500000  4766.180176  15644.969727  17.219999  

[731 rows x 10 columns]

In [34]:
data[["Close"]]

Close                                                   
                 BTC-USD         GC=F        ^GSPC         ^IXIC       ^VIX
Date                                                                       
2021-01-01  29374.152344          NaN          NaN           NaN        NaN
2021-01-02  32127.267578          NaN          NaN           NaN        NaN
2021-01-03  32782.023438          NaN          NaN           NaN        NaN
2021-01-04  31971.914062  1944.699951  3700.649902  12698.450195  26.969999
2021-01-05  33992.429688  1952.699951  3726.860107  12818.959961  25.340000
...                  ...          ...          ...           ...        ...
2021-12-27  50640.417969  1808.099976  4791.189941  15871.259766  17.680000
2021-12-28  47588.855469  1810.199951  4786.350098  15781.719727  17.540001
2021-12-29  46444.710938  1805.099976  4793.060059  15766.219727  16.950001
2021-12-30  47178.125000  1812.699951  4778.729980  15741.559570  17.330000
2021-12-31  46306.445312  1827.500000  4766.180176  15644.969727  17.219999

[365 rows x 5 columns]

In [32]:
# Убираем индех и переносим его в отдельную колонку для выстраивания в дальнейшем временные ряды
df1['Date'] = df1.index
df2 = df1.reindex(index=None)
values = {('Volume', 'GC=F'): 150} # Для примера поставил значение объема продаж золота 150 унций в день в колонке,
# где отсутствовали данные из-за неторговых дней.
df3 = df2.fillna(value=values)
df4 = df3.fillna(df.mean())
df1

Volume                                            Close  \
                BTC-USD   GC=F         ^GSPC         ^IXIC       BTC-USD   
Date                                                                       
2020-01-01  18565664997    NaN           NaN           NaN   7200.174316   
2020-01-02  20802083465  214.0  3.458250e+09  2.848370e+09   6985.470215   
2020-01-03  28111481032  107.0  3.461290e+09  2.567400e+09   7344.884277   
2020-01-04  18444271275    NaN           NaN           NaN   7410.656738   
2020-01-05  19725074095    NaN           NaN           NaN   7411.317383   
...                 ...    ...           ...           ...           ...   
2021-12-27  24324345758  150.0  2.264120e+09  3.730120e+09  50640.417969   
2021-12-28  33430376883  146.0  2.217050e+09  3.623600e+09  47588.855469   
2021-12-29  30049226299  623.0  2.369370e+09  3.694500e+09  46444.710938   
2021-12-30  26686491018  306.0  2.390990e+09  3.732730e+09  47178.125000   
2021-12-31  36974172400   80.0  2.446190e+09  3.379850e+09  46306.445312   

                                                                    Date  
                   GC=F        ^GSPC         ^IXIC       ^VIX             
Date                                                                      
2020-01-01          NaN          NaN           NaN        NaN 2020-01-01  
2020-01-02  1524.500000  3257.850098   9092.190430  12.470000 2020-01-02  
2020-01-03  1549.199951  3234.850098   9020.769531  14.020000 2020-01-03  
2020-01-04          NaN          NaN           NaN        NaN 2020-01-04  
2020-01-05          NaN          NaN           NaN        NaN 2020-01-05  
...                 ...          ...           ...        ...        ...  
2021-12-27  1808.099976  4791.189941  15871.259766  17.680000 2021-12-27  
2021-12-28  1810.199951  4786.350098  15781.719727  17.540001 2021-12-28  
2021-12-29  1805.099976  4793.060059  15766.219727  16.950001 2021-12-29  
2021-12-30  1812.699951  4778.729980  15741.559570  17.330000 2021-12-30  
2021-12-31  1827.500000  4766.180176  15644.969727  17.219999 2021-12-31  

[731 rows x 10 columns]

In [81]:
# combine store and item column as time_series**
df1['store'] = ['store_' + str(i) for i in df1['store']]
df1['item'] = ['item_' + str(i) for i in df1['item']]
df1['time_series'] = df1[['store', 'item']].apply(lambda x: '_'.join(x), axis=1)
df1.drop(['store', 'item'], axis=1, inplace=True)


KeyError: 'store'

In [ ]:

# extract features from date**
data['month'] = [i.month for i in data['date']]
data['year'] = [i.year for i in data['date']]
data['day_of_week'] = [i.dayofweek for i in data['date']]
data['day_of_year'] = [i.dayofyear for i in data['date']]

data.head()